# Imports

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px 
import seaborn as sns

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd


%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

# Data cleaning

In [18]:
# Import the data
df = pd.read_csv('NYPD_Complaint_Data_Current.csv')
df.head()

/var/folders/rv/xy288w3d3sz5n6227tysh3nr0000gn/T/ipykernel_3545/3273978596.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('NYPD_Complaint_Data_Current.csv')


,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,280510523,14,MANHATTAN,01/11/2024,04:00:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,104,FELONY,INSIDE,RAPE,(null),PATROL BORO MAN SOUTH,157.0,RAPE 1,HOTEL/MOTEL,01/14/2024,(null),18-24,BLACK,U,NaN,18-24,BLACK,F,985764.0,213806.0,40.753533,-73.994537,"(40.7535327012632, -73.9945368920152)",POINT (-73.9945368920152 40.7535327012632)
1,286736978,122,STATEN ISLAND,01/01/2015,13:00:00,01/01/2015,15:00:00,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,233,MISDEMEANOR,INSIDE,SEX CRIMES,(null),PATROL BORO STATEN ISLAND,175.0,"SEXUAL ABUSE 3,2",RESIDENCE-HOUSE,05/11/2024,(null),25-44,UNKNOWN,M,NaN,25-44,UNKNOWN,F,954827.0,148519.0,40.574286,-74.105914,"(40.5742856881925, -74.1059144051212)",POINT (-74.1059144051212 40.5742856881925)
2,289019347,1,MANHATTAN,06/20/2024,16:12:00,06/20/2024,16:13:00,COMPLETED,(null),NaN,1,N.Y. TRANSIT POLICE,578,VIOLATION,(null),HARRASSMENT 2,(null),PATROL BORO MAN SOUTH,637.0,"HARASSMENT,SUBD 1,CIVILIAN",TRANSIT - NYC SUBWAY,06/20/2024,FULTON STREET,45-64,BLACK,M,2.0,25-44,BLACK,F,982466.0,197787.0,40.709556,-74.006435,"(40.70955616028041, -74.00643483276845)",POINT (-74.00643483276845 40.70955616028041)
3,283688417,121,STATEN ISLAND,03/11/2024,10:00:00,03/13/2024,13:20:00,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,341,MISDEMEANOR,FRONT OF,PETIT LARCENY,(null),PATROL BORO STATEN ISLAND,339.0,"LARCENY,PETIT FROM OPEN AREAS,",MAILBOX OUTSIDE,03/13/2024,(null),(null),(null),(null),NaN,25-44,BLACK,F,938414.0,170500.0,40.634542,-74.165142,"(40.634542, -74.165142)",POINT (-74.165142 40.634542)
4,280934552,122,STATEN ISLAND,11/14/2023,12:00:00,11/17/2023,12:00:00,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,109,FELONY,INSIDE,GRAND LARCENY,(null),PATROL BORO STATEN ISLAND,418.0,"LARCENY,GRAND BY ACQUIRING LOST CREDIT CARD",RESIDENCE-HOUSE,01/22/2024,(null),(null),(null),(null),NaN,65+,WHITE,M,941085.0,135276.0,40.537874,-74.155293,"(40.537874, -74.155293)",POINT (-74.155293 40.537874)


In [19]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ADDR_PCT_CD,282069.0,6.413476e+01,35.134144,1.000000,40.000000,6.300000e+01,1.020000e+02,1.230000e+02
HOUSING_PSA,17650.0,6.113107e+03,12957.660677,218.000000,463.000000,6.720000e+02,1.233000e+03,7.277000e+04
JURISDICTION_CODE,282069.0,5.569347e-01,5.176240,0.000000,0.000000,0.000000e+00,0.000000e+00,9.700000e+01
KY_CD,282069.0,3.014404e+02,157.936083,101.000000,117.000000,3.410000e+02,3.510000e+02,8.810000e+02
PD_CD,281897.0,4.221454e+02,231.433171,100.000000,258.000000,3.860000e+02,6.380000e+02,9.690000e+02
TRANSIT_DISTRICT,12448.0,1.628205e+01,12.643224,1.000000,3.000000,1.200000e+01,3.200000e+01,3.400000e+01
X_COORD_CD,282065.0,1.005466e+06,21712.399561,0.000000,991271.000000,1.005028e+06,1.017949e+06,1.067306e+06
Y_COORD_CD,282065.0,2.075493e+05,29940.044035,0.000000,185346.000000,2.070650e+05,2.349010e+05,2.718190e+05
Latitude,282065.0,4.073602e+01,0.136080,0.000000,40.675332,4.073495e+01,4.081139e+01,4.091271e+01
Longitude,282065.0,-7.392286e+01,0.211627,-74.254555,-73.974664,-7.392496e+01,-7.387833e+01,0.000000e+00


In [20]:
weather_data = pd.read_csv('NYweather.csv', parse_dates=['datetime'])


In [21]:
crime_data = pd.read_csv('NYPD_Complaint_Data_Current.csv')
weather_data = pd.read_csv('NYweather.csv')

# Convert 'CMPLNT_FR_DT' in crime_data to datetime format with error handling
crime_data['CMPLNT_FR_DT'] = pd.to_datetime(crime_data['CMPLNT_FR_DT'], format='%m/%d/%Y', errors='coerce').dt.date

# Drop rows with invalid 'CMPLNT_FR_DT'
crime_data = crime_data.dropna(subset=['CMPLNT_FR_DT'])

# Filter between 2024-01-01 and 2024-10-01
start_date = pd.to_datetime('2024-01-01').date()
end_date = pd.to_datetime('2024-10-01').date()
crime_data = crime_data[(crime_data['CMPLNT_FR_DT'] >= start_date) & (crime_data['CMPLNT_FR_DT'] <= end_date)]

# Convert 'datetime' in weather_data to datetime format
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], format='%Y-%m-%d').dt.date

# Merge crime_data with weather_data on the date
crime_weather_data = pd.merge(crime_data, weather_data, how='left', left_on='CMPLNT_FR_DT', right_on='datetime')

# Drop unnecessary 'datetime' column from weather_data
crime_weather_data = crime_weather_data.drop(columns=['datetime'])

# Save the combined data
crime_weather_data.to_csv('Crime_Weather_Combined.csv', index=False)

crime_weather_data.head()

/var/folders/rv/xy288w3d3sz5n6227tysh3nr0000gn/T/ipykernel_3545/337948210.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_data = pd.read_csv('NYPD_Complaint_Data_Current.csv')


,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column,name,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,280510523,14,MANHATTAN,2024-01-11,04:00:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,104,FELONY,INSIDE,RAPE,(null),PATROL BORO MAN SOUTH,157.0,RAPE 1,HOTEL/MOTEL,01/14/2024,(null),18-24,BLACK,U,NaN,18-24,BLACK,F,985764.0,213806.0,40.753533,-73.994537,"(40.7535327012632, -73.9945368920152)",POINT (-73.9945368920152 40.7535327012632),newyork,25.0,16.0,18.9,25.0,16.0,18.9,15.5,81.9,0.0,0,0.0,NaN,0,0,16.6,13.0,59.3,1018.6,21.1,4.3,197.7,17.0,7,10,2024-01-11T07:08:58,2024-01-11T17:59:45,0.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"42752099999,42662099999,42754099999,remote,VAID"
1,289019347,1,MANHATTAN,2024-06-20,16:12:00,06/20/2024,16:13:00,COMPLETED,(null),NaN,1,N.Y. TRANSIT POLICE,578,VIOLATION,(null),HARRASSMENT 2,(null),PATROL BORO MAN SOUTH,637.0,"HARASSMENT,SUBD 1,CIVILIAN",TRANSIT - NYC SUBWAY,06/20/2024,FULTON STREET,45-64,BLACK,M,2.0,25-44,BLACK,F,982466.0,197787.0,40.709556,-74.006435,"(40.70955616028041, -74.00643483276845)",POINT (-74.00643483276845 40.70955616028041),newyork,35.0,24.0,28.9,39.1,24.0,31.2,21.9,67.4,0.0,0,0.0,rain,0,0,36.4,24.1,247.8,1002.7,42.2,5.7,212.8,18.2,9,30,2024-06-20T05:42:39,2024-06-20T19:14:00,0.44,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"42752099999,42662099999,42754099999,VAID"
2,283688417,121,STATEN ISLAND,2024-03-11,10:00:00,03/13/2024,13:20:00,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,341,MISDEMEANOR,FRONT OF,PETIT LARCENY,(null),PATROL BORO STATEN ISLAND,339.0,"LARCENY,PETIT FROM OPEN AREAS,",MAILBOX OUTSIDE,03/13/2024,(null),(null),(null),(null),NaN,25-44,BLACK,F,938414.0,170500.0,40.634542,-74.165142,"(40.634542, -74.165142)",POINT (-74.165142 40.634542),newyork,33.2,15.0,25.2,32.0,15.0,24.7,9.8,39.6,0.0,0,0.0,NaN,0,0,22.7,14.8,299.9,1017.1,0.0,6.4,279.1,24.0,9,10,2024-03-11T06:38:56,2024-03-11T18:34:31,0.03,Clear,Clear conditions throughout the day.,clear-day,"42752099999,42662099999,42754099999,remote,VAID"
3,281033034,72,BROOKLYN,2024-01-23,19:00:00,01/24/2024,07:15:00,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,121,FELONY,(null),CRIMINAL MISCHIEF & RELATED OF,(null),PATROL BORO BKLYN SOUTH,267.0,"MISCHIEF, CRIMINAL 3 & 2, OF M",STREET,01/24/2024,(null),UNKNOWN,UNKNOWN,U,NaN,25-44,WHITE,F,983763.0,179737.0,40.660013,-74.001755,"(40.660013201115774, -74.00175528932395)",POINT (-74.00175528932395 40.660013201115774),newyork,23.2,8.0,16.3,23.2,8.0,16.3,4.7,48.4,0.0,0,0.0,NaN,0,0,19.8,22.3,41.7,1018.1,13.5,6.7,221.0,19.1,8,10,2024-01-23T07:08:36,2024-01-23T18:08:13,0.41,Clear,Clear conditions throughout the day.,clear-day,"42752099999,42662099999,42754099999,remote,VAID"
4,288178776,68,BROOKLYN,2024-05-28,10:00:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,341,MISDEMEANOR,(null),PETIT LARCENY,(null),PATROL BORO BKLYN SOUTH,353.0,"LARCENY,PETIT FROM BUILDING,UNATTENDED, PACKAG...",RESIDENCE-HOUSE,06/05/2024,(null),(null),(null),(null),NaN,25-44,ASIAN / PACIFIC ISLANDER,M,975766.0,165541.0,40.621044,-74.030559,"(40.621044, -74.030559)",POINT (-74.030559 40.621044),newyork,40.3,27.0,33.4,43.7,29.2,36.6,20.9,50.3,0.0,0,0.0,NaN,0,0,67.0,38.0,274.2,1001.4,9.3,6.4,334.5,29.0,10,10,2024-05-28T05:42:13,2024-05-28T19:05:57,0.67,Clear,Clear conditions throughout the day.,clear-day,"42752099999,42662099999,427540

In [16]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Initialize the Nominatim object for reverse geocoding
geolocator = Nominatim(user_agent="crime_weather_app")

# Add a rate limiter to avoid hitting usage limits
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Function to get zipcode based on latitude and longitude
def get_zipcode(latitude, longitude):
    if pd.notnull(latitude) and pd.notnull(longitude):
        location = reverse((latitude, longitude), exactly_one=True)
        if location and 'postcode' in location.raw['address']:
            return location.raw['address']['postcode']
    return None

# Apply the function to each row in the dataframe to create a new column 'zipcode'
crime_weather_data['zipcode'] = crime_weather_data.apply(lambda row: get_zipcode(row['Latitude'], row['Longitude']), axis=1)


crime_weather_data.head()

RateLimiter caught an error, retrying (0/2 tries). Called with (*((40.717792, -74.01061),), **{'exactly_one': True}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py", line 461, in _make_request
    httplib_response = conn.getresponse()
                       ^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1378, in getresponse
    response.begin()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/l

KeyboardInterrupt: 

In [5]:
# drop useless columns
df.drop(columns = ['PD_CD','PD_DESC','JURISDICTION_CODE','JURIS_DESC','KY_CD','RPT_DT','LOC_OF_OCCUR_DESC'],inplace = True)

In [6]:
# percentage of nan values for each column
for column in df.columns:
    print(f'{column}:missing {df[column].isna().sum()*100/len(df)}')

CMPLNT_NUM:missing 0.0
ADDR_PCT_CD:missing 0.0
BORO_NM:missing 0.0
CMPLNT_FR_DT:missing 0.0
CMPLNT_FR_TM:missing 0.0
CMPLNT_TO_DT:missing 5.276368548121204
CMPLNT_TO_TM:missing 0.0
CRM_ATPT_CPTD_CD:missing 0.0
HADEVELOPT:missing 0.0
HOUSING_PSA:missing 93.7426658016301
LAW_CAT_CD:missing 0.0
OFNS_DESC:missing 0.0
PARKS_NM:missing 0.0
PATROL_BORO:missing 0.0
PREM_TYP_DESC:missing 0.0
STATION_NAME:missing 0.0
SUSP_AGE_GROUP:missing 0.0
SUSP_RACE:missing 0.0
SUSP_SEX:missing 0.0
TRANSIT_DISTRICT:missing 95.58689540502502
VIC_AGE_GROUP:missing 0.0
VIC_RACE:missing 0.0
VIC_SEX:missing 0.0
X_COORD_CD:missing 0.0014180927361744822
Y_COORD_CD:missing 0.0014180927361744822
Latitude:missing 0.0014180927361744822
Longitude:missing 0.0014180927361744822
Lat_Lon:missing 0.0014180927361744822
New Georeferenced Column:missing 0.0014180927361744822


In [7]:
# drop columns with high nan and redundent information %
df.drop(columns=['HOUSING_PSA','TRANSIT_DISTRICT'],inplace=True)

In [ ]:
df.loc[df['PARKS_NM'].notnull(),'PARKS_NM'] = 1
df.loc[df['PARKS_NM'].isna(),'PARKS_NM'] = 0

df.loc[df['HADEVELOPT'].notnull(),'HADEVELOPT'] = 1
df.loc[df['HADEVELOPT'].isna(),'HADEVELOPT'] = 0

df.loc[df['STATION_NAME'].notnull(),'STATION_NAME'] = 1
df.loc[df['STATION_NAME'].isna(),'STATION_NAME'] = 0


df = df.rename({'PARKS_NM':'IN_PARK','HADEVELOPT':'IN_PUBLIC_HOUSING','STATION_NAME':'IN_STATION'},axis = 1)

In [ ]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Use rate limiter to avoid hitting the request limit
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geocode((df[lat_field], df[lon_field]))
    return location.address.split(',')[-2].strip() if location else None

# Test the function with a single request
print(get_zipcode({'Latitude': 40.712776, 'Longitude': -74.005974}, geolocator, 'Latitude', 'Longitude'))

RateLimiter caught an error, retrying (0/2 tries). Called with (*((40.712776, -74.005974),), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopy/geocoders/base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopy/adapters.py", line 472, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopy/adapters.py", line 500, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 403

The above exception was the direct cause of the following exception:

Traceback (most 

None


In [17]:
#remove unnecessary coordinate columns
df.drop(columns=['X_COORD_CD','Y_COORD_CD',"Lat_Lon"],inplace=True)

In [18]:
# Rows without coordinates should be dropped
df = df[df['Latitude'].notna()]
df = df[df['Longitude'].notna()]

In [19]:
# check if the two columns have the same values
if all(df['CMPLNT_FR_DT'].notna() == df['CMPLNT_TO_DT'].notna()):
    df.drop(columns='CMPLNT_TO_DT',inplace = True)

In [20]:
# change date columns to YYYY-MM-DD
df['CMPLNT_FR_DT']= pd.to_datetime(df['CMPLNT_FR_DT'],errors='coerce')
df['CMPLNT_FR_TM'] = pd.to_datetime(df['CMPLNT_FR_TM'],errors='coerce')

/var/folders/rv/xy288w3d3sz5n6227tysh3nr0000gn/T/ipykernel_66298/2903593181.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CMPLNT_FR_TM'] = pd.to_datetime(df['CMPLNT_FR_TM'],errors='coerce')


In [21]:
# delete columns that have no event start date or time
df = df[df['CMPLNT_FR_DT'].notna()]
df = df[df['CMPLNT_FR_TM'].notna()]

In [22]:
# these columns have redundent info
df.drop(columns='CMPLNT_TO_TM',inplace = True)
df.drop(columns='CMPLNT_TO_DT',inplace = True)

In [23]:
# split date to year, month and day
df['year'] = df['CMPLNT_FR_DT'].dt.year
df['month'] = df['CMPLNT_FR_DT'].dt.month
df['day'] = df['CMPLNT_FR_DT'].dt.day
df['hour'] =df['CMPLNT_FR_TM'].dt.hour
# add week day column
df['weekday'] = pd.to_datetime(df['CMPLNT_FR_DT']).dt.day_name()
df.drop(columns='CMPLNT_FR_DT',inplace = True)
df.drop(columns='CMPLNT_FR_TM',inplace = True)

In [24]:
df.head()

,CMPLNT_NUM,ADDR_PCT_CD,OFNS_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,PREM_TYP_DESC,IN_PARK,IN_PUBLIC_HOUSING,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,PATROL_BORO,IN_STATION,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,year,month,day,hour,weekday
0,39468181,88.0,BURGLARY,COMPLETED,FELONY,BROOKLYN,RESIDENCE - APT. HOUSE,1,1,(null),UNKNOWN,(null),40.692464,-73.972708,PATROL BORO BKLYN NORTH,1,25-44,WHITE,F,2008,2,20,7,Wednesday
1,50539499,19.0,GRAND LARCENY,COMPLETED,FELONY,MANHATTAN,BAR/NIGHT CLUB,1,1,(null),(null),(null),40.771341,-73.953418,PATROL BORO MAN NORTH,1,45-64,WHITE HISPANIC,F,2008,8,21,22,Thursday
2,45223390,77.0,FELONY ASSAULT,COMPLETED,FELONY,BROOKLYN,RESIDENCE - PUBLIC HOUSING,1,1,(null),(null),(null),40.671245,-73.926713,PATROL BORO BKLYN NORTH,1,25-44,BLACK,F,2008,4,3,3,Thursday
3,50594658,32.0,PETIT LARCENY,COMPLETED,MISDEMEANOR,MANHATTAN,STREET,1,1,(null),UNKNOWN,(null),40.813412,-73.943226,PATROL BORO MAN NORTH,1,(null),UNKNOWN,M,2008,8,19,9,Tuesday
4,44451016,67.0,ROBBERY,COMPLETED,FELONY,BROOKLYN,STREET,1,1,<18,BLACK,M,40.650142,-73.944674,PATROL BORO BKLYN SOUTH,1,25-44,BLACK,M,2008,3,10,22,Monday


In [25]:
# clean suspect and victim data
df['SUSP_RACE'] = df['SUSP_RACE'].fillna('UNKNOWN')
df['VIC_RACE'] = df['VIC_RACE'].fillna('UNKNOWN')

df['SUSP_SEX'] = df['SUSP_SEX'].fillna('U')
df['VIC_SEX'] = df['VIC_SEX'].fillna('U')

In [26]:
# take care of unvalid age data
valid_age = ['UNKNOWN', '25-44', '<18', '45-64', '65+', '18-24']
df.loc[~df['SUSP_AGE_GROUP'].isin(valid_age),'SUSP_AGE_GROUP'] = 'UNKNOWN'
df.loc[~df['VIC_AGE_GROUP'].isin(valid_age),'VIC_AGE_GROUP'] = 'UNKNOWN'

In [27]:
# redundent data
df.drop(columns='PATROL_BORO',inplace = True)
df['BORO_NM'] = df['BORO_NM'].fillna('UNKNOWN')

In [28]:
df.head()

,CMPLNT_NUM,ADDR_PCT_CD,OFNS_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,PREM_TYP_DESC,IN_PARK,IN_PUBLIC_HOUSING,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,IN_STATION,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,year,month,day,hour,weekday
0,39468181,88.0,BURGLARY,COMPLETED,FELONY,BROOKLYN,RESIDENCE - APT. HOUSE,1,1,UNKNOWN,UNKNOWN,(null),40.692464,-73.972708,1,25-44,WHITE,F,2008,2,20,7,Wednesday
1,50539499,19.0,GRAND LARCENY,COMPLETED,FELONY,MANHATTAN,BAR/NIGHT CLUB,1,1,UNKNOWN,(null),(null),40.771341,-73.953418,1,45-64,WHITE HISPANIC,F,2008,8,21,22,Thursday
2,45223390,77.0,FELONY ASSAULT,COMPLETED,FELONY,BROOKLYN,RESIDENCE - PUBLIC HOUSING,1,1,UNKNOWN,(null),(null),40.671245,-73.926713,1,25-44,BLACK,F,2008,4,3,3,Thursday
3,50594658,32.0,PETIT LARCENY,COMPLETED,MISDEMEANOR,MANHATTAN,STREET,1,1,UNKNOWN,UNKNOWN,(null),40.813412,-73.943226,1,UNKNOWN,UNKNOWN,M,2008,8,19,9,Tuesday
4,44451016,67.0,ROBBERY,COMPLETED,FELONY,BROOKLYN,STREET,1,1,<18,BLACK,M,40.650142,-73.944674,1,25-44,BLACK,M,2008,3,10,22,Monday


In [29]:
df.dropna(inplace=True)
len(df)

8913192

In [30]:
# encode crime severity
df['LAW_CAT_CD'] = df['LAW_CAT_CD'].replace(['felony','misdemeanor','violation'],[2,1,0])

In [31]:
df.columns

Index(['CMPLNT_NUM', 'ADDR_PCT_CD', 'OFNS_DESC', 'CRM_ATPT_CPTD_CD',
       'LAW_CAT_CD', 'BORO_NM', 'PREM_TYP_DESC', 'IN_PARK',
       'IN_PUBLIC_HOUSING', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX',
       'Latitude', 'Longitude', 'IN_STATION', 'VIC_AGE_GROUP', 'VIC_RACE',
       'VIC_SEX', 'year', 'month', 'day', 'hour', 'weekday'],
      dtype='object')

In [32]:

df = df.rename({'LAW_CAT_CD':'CRIME_CLASS'},axis = 1)


In [33]:
df['CRIME_CLASS'].value_counts()

CRIME_CLASS
MISDEMEANOR    4915701
FELONY         2791935
VIOLATION      1205556
Name: count, dtype: int64

In [34]:
column_titles = ['CMPLNT_NUM','year','month', 'day','weekday','hour','Latitude', 'Longitude', 'CRM_ATPT_CPTD_CD',
                 'OFNS_DESC', 'ADDR_PCT_CD', 'CRIME_CLASS', 'BORO_NM', 'PREM_TYP_DESC',
                 'IN_PARK', 'IN_PUBLIC_HOUSING', 'IN_STATION',
                 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX',
                 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']
df = df.reindex(columns = column_titles)

In [35]:
df.head()

,CMPLNT_NUM,year,month,day,weekday,hour,Latitude,Longitude,CRM_ATPT_CPTD_CD,OFNS_DESC,ADDR_PCT_CD,CRIME_CLASS,BORO_NM,PREM_TYP_DESC,IN_PARK,IN_PUBLIC_HOUSING,IN_STATION,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,39468181,2008,2,20,Wednesday,7,40.692464,-73.972708,COMPLETED,BURGLARY,88.0,FELONY,BROOKLYN,RESIDENCE - APT. HOUSE,1,1,1,UNKNOWN,UNKNOWN,(null),25-44,WHITE,F
1,50539499,2008,8,21,Thursday,22,40.771341,-73.953418,COMPLETED,GRAND LARCENY,19.0,FELONY,MANHATTAN,BAR/NIGHT CLUB,1,1,1,UNKNOWN,(null),(null),45-64,WHITE HISPANIC,F
2,45223390,2008,4,3,Thursday,3,40.671245,-73.926713,COMPLETED,FELONY ASSAULT,77.0,FELONY,BROOKLYN,RESIDENCE - PUBLIC HOUSING,1,1,1,UNKNOWN,(null),(null),25-44,BLACK,F
3,50594658,2008,8,19,Tuesday,9,40.813412,-73.943226,COMPLETED,PETIT LARCENY,32.0,MISDEMEANOR,MANHATTAN,STREET,1,1,1,UNKNOWN,UNKNOWN,(null),UNKNOWN,UNKNOWN,M
4,44451016,2008,3,10,Monday,22,40.650142,-73.944674,COMPLETED,ROBBERY,67.0,FELONY,BROOKLYN,STREET,1,1,1,<18,BLACK,M,25-44,BLACK,M


In [36]:
df = df[df['year']>2005]
df_2019 = df[df['year']==2019]
df_train = df[df['year']!=2019]

In [ ]:
df.to_csv('ny_clean_all.csv',index= False)
df_train.to_csv('ny_clean.csv',index=False)
df_2019.to_csv('ny_clean_2019.csv',index=False)